In [ ]:
import json

with open("distinct_models.json") as f:
    distinct_models = json.load(f)


with open("test.json") as f:
    all_cars = json.load(f)

from_all = set([c["build"]["model"] for c in all_cars])
from_all

In [ ]:
import json
from backend.db.products import product_db, Product, VehicleType, Powertrain, Color
from pprint import pprint
import traceback


# Problem with normalization of Mini MPV
def normalize(value):
    return "Mini MPV" if value == "Mini Mpv" else value


product_list = []
for model in from_all:
    for car in all_cars:
        if car["build"]["model"] == model:
            try:
                product = Product(
                    name=car["heading"],
                    vehicle_type=VehicleType(normalize(car["build"]["body_type"])),
                    release_year=car["build"]["year"],
                    price_in_usd=car.get("price")
                    or car.get("ref_price")
                    or car.get("msrp"),
                    is_used=car["inventory_type"] == "used",
                    powertrain_type=Powertrain[
                        "EV"
                        if car["build"].get("powertrain_type") == "BEV"
                        else "ICE"
                        if car["build"].get("powertrain_type") == "Combustion"
                        else "Hybrid"
                    ],
                    num_seats=car["build"]["std_seating"],
                    color=car.get("base_ext_color"),
                    specs=car["build"],
                    image_url=car["media"]["photo_links"][:1]
                    if car.get("media")
                    else [],
                )

                # override image from distinct_models
                for m in distinct_models:
                    if m["model"] == model:
                        better_img = m["links"][:1]
                        if better_img:
                            print("overriding image")
                            product.image_url = better_img
                            break

            except Exception as e:
                # print("skipping car")
                # traceback.print_exc()
                # print("Error in parsing the following listing:")
                # pprint(car)
                continue

    product_list.append(product)

In [ ]:
product_list

In [ ]:
# display the images in jupyter
from IPython.display import Image, display

for product in product_list:
    print(product.image_url)
    display(Image(url=product.image_url[0]))

In [ ]:
# save to db
await product_db.reset()
await product_db.add(product_list)

In [ ]:
await product_db.size()